# Push final models to Huggingface

**Purpose:** This script is used to push final models to huggingface model-hub. This can also be done manually. 

**Dependency:** Prior to running this script, final models need to be trained (`Train_final_models.ipynb`) and downloaded from weights and biases and placed in the `TRIDENT`-folder.

**Consecutive scripts:** After running this script the following scripts may be executed. `download_wandb_artifacts.ipynb`

In [1]:
import transformers
from transformers import AutoModel, AutoTokenizer

import torch

from development_utils.training.Build_Pytorch_model import TRIDENT, DNN_module

In [2]:
model_path = '../TRIDENT/'
version = 'EC50_fish'
name = f'final_model_{version}'

In [3]:
onehotencodinglengths = {
    'EC50_algae': 1,
    'EC10_algae': 1,
    'EC50EC10_algae': 2, 
    'EC50_invertebrates': 2,
    'EC10_invertebrates': 6,
    'EC50EC10_invertebrates': 8,
    'EC50_fish': 1,
    'EC10_fish': 7,
    'EC50EC10_fish': 9
}

In [4]:
chemberta = AutoModel.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')
tokenizer = AutoTokenizer.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')

Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

c:\Anaconda3\envs\TRIDENT_3\lib\site-packages\huggingface_hub-0.11.1-py3.8.egg\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Styrbjörn Käll\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/165k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/101k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [5]:
dnn = DNN_module(one_hot_enc_len=onehotencodinglengths[version],
                n_hidden_layers=3,
                layer_sizes=[700,500,300],
                dropout=0.2)

model = TRIDENT(chemberta, dnn)

In [6]:
def load_ckp(checkpoint_dir, model):
    checkpoint_dnn = torch.load(checkpoint_dir+'_dnn_saved_weights.pt', map_location='cpu')
    checkpoint_roberta = torch.load(checkpoint_dir+'_roberta_saved_weights.pt', map_location='cpu')
    model.dnn.load_state_dict(checkpoint_dnn)
    model.roberta.load_state_dict(checkpoint_roberta)
    return model

In [7]:
model = load_ckp(model_path+name, model)
model.eval()

TRIDENT(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

## Save to hub

In [8]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Styrbjörn Käll\.huggingface\token
Login successful


In [10]:
model.roberta.push_to_hub(version)

In [ ]:
tokenizer.push_to_hub(version)

CommitInfo(commit_url='https://huggingface.co/StyrbjornKall/EC10_fish/commit/fc411db6a4aaa87d194d30829d0c8fe6a605f6f1', commit_message='Upload tokenizer', commit_description='', oid='fc411db6a4aaa87d194d30829d0c8fe6a605f6f1', pr_url=None, pr_revision=None, pr_num=None)